# Emission Model Configuration
In this notebook, we'll configure the emission model to be used by our AR simulations. This will include many unresolved lines as well as a few resolved lines that will be observed by *Hinode*/EIS. The information regarding the resolved lines will be taken from Warren et al. (2012).

In [1]:
import os
import urllib
import io
import logging
logging.basicConfig(level=logging.INFO)

import pandas
import h5py
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
import ChiantiPy.core as ch
import ChiantiPy.tools.util as ch_tools_util
import ChiantiPy.tools.data as ch_data

from synthesizAR.atomic import EmissionModel,ChIon

%matplotlib inline

 using cli
 using CLI for selections
 reading chiantirc file


/home/wtb2/anaconda3/envs/systematic-ar-study/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
/home/wtb2/anaconda3/envs/systematic-ar-study/lib/python3.5/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


## Read in Data from Warren et al. (2012)

In [2]:
tmp = (urllib.request.urlopen('http://iopscience.iop.org/0004-637X/759/2/141/suppdata/apj446760t2_mrt.txt')
                     .readlines())

In [3]:
col_names = [str(t.strip()).split('  ')[-1] for t in tmp[8:15]]
col_names = [c[:-1] for c in col_names[0:2]] + ['Element','Ion','Wavelength'] + [c[:-1] for c in col_names[3:]]

In [4]:
table_io = io.StringIO(','.join(col_names) + '\n' 
                       + '\n'.join([','.join(filter(None,t.strip().decode('utf8').split(' '))) for t in tmp[19:]]))

In [5]:
df = pandas.read_csv(table_io)

In [6]:
df

,Active region,Instrument (1),Element,Ion,Wavelength,Observed intensity,Error in OInt,Differential emission measure intensity,Ratio of OInt to DInt
0,1,EIS,Si,VII,275.368,133.23,29.38,132.41,1.01
1,1,EIS,Fe,IX,188.497,143.82,31.74,144.22,1.00
2,1,EIS,Fe,X,184.536,463.08,102.04,406.30,1.14
3,1,EIS,Fe,XI,188.216,886.79,195.15,970.65,0.91
4,1,EIS,Fe,XII,192.394,659.39,145.10,767.61,0.86
5,1,EIS,Fe,XII,195.119,2154.79,474.08,2393.04,0.90
6,1,EIS,Fe,XIII,202.044,1949.12,428.88,1409.76,1.38
7,1,EIS,Fe,XIII,203.826,3223.85,709.42,2304.45,1.40
8,1,EIS,Fe,XV,284.160,10382.98,2284.38,12679.17,0.82
9,1,EIS,S,XIII,256.686,697.99,153.65,738.30,0.95


## Selecting Ions
Select the number of (unresolved) ions in addition to the resolved lines for the active region. For AR 9 and 10, we'll create distinct emission models based on the lines selected by Warren et al. (2012).

We need to correct each of the resolved wavelengths to make sure that they can be found in the CHIANTI database.

In [7]:
df_1109 = df[df['Active region']==9]

In [8]:
df_1109

,Active region,Instrument (1),Element,Ion,Wavelength,Observed intensity,Error in OInt,Differential emission measure intensity,Ratio of OInt to DInt
168,9,EIS,Si,VII,275.368,30.76,6.81,32.29,0.95
169,9,EIS,Fe,IX,188.497,46.43,10.26,46.55,1.00
170,9,EIS,Fe,IX,197.862,29.40,6.49,29.01,1.01
171,9,EIS,Fe,X,184.536,210.65,46.40,168.10,1.25
172,9,EIS,Fe,XI,180.401,749.76,165.15,891.80,0.84
173,9,EIS,Fe,XI,188.216,467.97,102.97,443.68,1.05
174,9,EIS,S,X,264.233,53.29,11.77,57.22,0.93
175,9,EIS,Si,X,258.375,218.87,48.19,272.98,0.80
176,9,EIS,Fe,XII,192.394,386.91,85.13,340.03,1.14
177,9,EIS,Fe,XII,195.119,1236.18,271.97,1060.09,1.17


In [9]:
resolved_lines_1109 = {}
for i,r in df_1109[df_1109['Instrument (1)']=='EIS'].iterrows():
    # parse ion name
    ion_name = ch_tools_util.spectroscopic2name(r['Element'],r['Ion'])
    print('Processing ion {}'.format(ion_name))
    # correct wavelength
    w = float(r['Wavelength'])
    tmp = ch.ion(ion_name)
    new_w = min(tmp.Wgfa['wvl'],key=lambda x:abs(x-w))
    if abs(new_w - w)>0.1:
        raise ValueError('Closest wavelength {} is not within tolerance of {}'.format(new_wvl,w))
    w = new_w
    # save
    if ion_name not in resolved_lines_1109:
        resolved_lines_1109[ion_name] = [w]
    else:
        resolved_lines_1109[ion_name].append(w)

Processing ion si_7
Processing ion fe_9
Processing ion fe_9
Processing ion fe_10
Processing ion fe_11
Processing ion fe_11
Processing ion s_10
Processing ion si_10
Processing ion fe_12
Processing ion fe_12
Processing ion fe_13
Processing ion fe_13
Processing ion fe_14
Processing ion fe_14
Processing ion fe_15
Processing ion s_13
Processing ion fe_16
Processing ion ar_14
Processing ion ca_14
Processing ion ca_15
Processing ion ca_16
Processing ion ca_17


~~We cannot calculate the emission for every ion in the CHIANTI database. This is far too time consuming and takes up way too much space on disk.~~ Instead what we'll do is calculate the emissivity for only a limited number of ions, at least including those whose wavelengths/transitions we are resolving. In this case, we'll calculate the emissivity for Fe, Si, S, Ca, and Ar.

Nevermind. We'll try to do it for all the ions.

In [10]:
all_ion_list_1109 = []
for ion in ch_data.MasterList:
    if ion[-1]=='d': #or ion.split('_')[0] not in set([k.split('_')[0] for k in resolved_lines_1109]):
        continue
    tmp = {'name':ion,
           'resolved_wavelengths':[]
          }
    if ion in resolved_lines_1109:
        tmp['resolved_wavelengths'] = resolved_lines_1109[ion]
    tmp['resolved_wavelengths'] = tmp['resolved_wavelengths']*u.angstrom
    all_ion_list_1109.append(tmp)

## Create Emission Model
Now, pass the complete ion list to the emission model. In this way, we've told the emission model to only calculate the wavelength-resolved emission from those ions we picked from the Warren et al. list. We're calculating the 

In [11]:
emission_model_1109 = EmissionModel(all_ion_list_1109,
                                    temperature=np.logspace(4.5,8,50)*u.K,
                                    density=np.logspace(7,11,20)/(u.cm**3),
                                    energy_unit='photon',
                                    chianti_db_filename='/data/datadrive1/ar_forward_modeling/systematic_ar_study/chianti_db.h5'
                                   )

INFO:EmissionModel:Using CHIANTI HDF5 database in /data/datadrive1/ar_forward_modeling/systematic_ar_study/chianti_db.h5
INFO:EmissionModel:Building entry for h_1
INFO:EmissionModel:Building elvlc entry for h_1
INFO:EmissionModel:Building wgfa entry for h_1
INFO:EmissionModel:Building scups entry for h_1
INFO:EmissionModel:Building psplups entry for h_1
INFO:EmissionModel:Building entry for he_2
INFO:EmissionModel:Building elvlc entry for he_2
INFO:EmissionModel:Building wgfa entry for he_2
INFO:EmissionModel:Building scups entry for he_2
INFO:EmissionModel:Building psplups entry for he_2
INFO:EmissionModel:Building entry for c_6
INFO:EmissionModel:Building elvlc entry for c_6
INFO:EmissionModel:Building wgfa entry for c_6
INFO:EmissionModel:Building scups entry for c_6
INFO:EmissionModel:Building psplups entry for c_6
INFO:EmissionModel:Building entry for n_7
INFO:EmissionModel:Building elvlc entry for n_7
INFO:EmissionModel:Building wgfa entry for n_7
INFO:EmissionModel:Building scup

Calculate emissivity.

In [12]:
emission_model_1109.calculate_emissivity()

INFO:EmissionModel:Calculating emissivity for ion h_1
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion he_2
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion c_6
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion n_7
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion o_8
INFO:ChIon:Expressing emissivity in units of photons
INFO:ChIon:Calculating level populations.
INFO:ChIon:Calculating emissivity
INFO:EmissionModel:Calculating emissivity for ion ne_10
INFO:ChIon:Expressing emissi

Save the emissivity model.

In [13]:
emission_model_1109.save(savedir='/data/datadrive1/ar_forward_modeling/systematic_ar_study/emission_model1109_full')

INFO:EmissionModel:Saving emission model information in /data/datadrive1/ar_forward_modeling/systematic_ar_study/emission_model1109_full
